# Simulation Wrapper

5/27/21

#### Summary:
Run a crude version of system realization and latency measurements given the offloading decisions are identical for user to assumptions in container dispatchment.
- Initialize the system (servers, users, containers)
- Deploy the containers for one cycle
- Offload user jobs and record lateny throughout the system 
- Move the users based on their markovian mobility
- Reassign containers and repeat

In [1]:
cd /home/tkim/Academics/DistMEC

/home/tkim/Academics/DistMEC


### Import Relevant Libraries and Dependencies

In [36]:
import numpy as np
import math
import copy

from classes.Application import *
from classes.User import *
from classes.Server import *
from solver.Sim_Params import *
from classes.Central_Controller import *
from classes.Container import *

### Build Wrapper Class

The wrapper class should propagate between small and big timesteps.

Functions should take charge of each step of the cycle

In [37]:
class Sim_Params:
    """
    Simulation params hold information about system setting for simulation
    - timestep - 5 min per timestep
    - length - 1 mile per unit length
    """
    
    def __init__(self, big_ts, small_ts, x_length, y_length, num_users, num_servers, num_apps):
        
        self.big_ts = big_ts
        self.small_ts = small_ts
        self.x_length = x_length
        self.y_length = y_length
        self.num_users = num_users
        self.num_servers = num_servers
        self.num_apps = num_apps
        
        # Non-specified instances
        self.low_mean_jobs = 5
        self.high_mean_jobs = 15
        self.server_weak_range = np.array([[1,4]])
        self.server_strong_range = np.array([[1,4]])
        self.user_max_speed = 2.5
        self.user_lamdas = [1/0.25,1/0.83,1/1.67] # 3 mph, 10 mph, 20 mph
        self.user_num_path = 10
        self.container_service_low = 20
        self.container_service_high = 30
        self.deploy_rounds = 5

In [38]:
def setup_sim(sim_param):
    
    # Create Job Profiles
    num_app_types = sim_param.num_apps
    low_mean = sim_param.low_mean_jobs
    high_mean = sim_param.high_mean_jobs
    job_profiles = []

    for i in range(num_app_types):
        job_profiles += [Job_Profile(job_name = str(i),
                                     latency_req = 3,
                                     offload_mean = np.random.uniform(low_mean,high_mean))]


    # System physical Boundaries - All action takes within this
    boundaries = np.array([[0,sim_param.x_length],[0,sim_param.y_length]])


    # Generate Servers
    num_resource = 1
    weak_range = sim_param.server_weak_range
    strong_range = sim_param.server_strong_range

    # Generate Server
    servers = []
    idx_counter = 0

    for i in range(sim_param.num_servers):
        servers.append(Server(boundaries,level=2,rand_locs=True,locs=None))
        servers[-1].server_resources(num_resource, weak_range, strong_range)
        servers[-1].assign_id(idx_counter)
        idx_counter += 1


    # Generate Users
    users= []
    idx_counter = 0


    for i in range(sim_param.num_users):
        users += [User(boundaries, sim_param.big_ts, 2, sim_param.user_lamdas, sim_param.user_max_speed)]
        users[-1].generate_MC(servers)
        users[-1].assign_id(idx_counter)
        idx_counter += 1


    # Generate Apps
    num_apps = len(users)
    app_id = np.random.choice(num_app_types,num_apps)
    apps = []

    for i in range(len(app_id)):
        apps += [Application(job_type=app_id[i], user_id=i, 
                             time_steps=sim_param.big_ts, job_profiles=job_profiles)]
        
    # Generate Containers - in dictionary indexed by {(server,app)}
    containers = {}
    
    for s in range(len(servers)):
        for a in range(num_app_types):
            service_rate = np.random.uniform(sim_param.container_service_low, sim_param.container_service_high)
            latency_restriction = job_profiles[a].latency_req
            containers[(s,a)] = Container(a, s, service_rate, latency_restriction)

    return servers, users, containers, apps

### Run the system

First set up simulation parameters and generate players (servers, users, containers, apps)

In [39]:
sim_param = Sim_Params(big_ts=5, small_ts=5, x_length=5, y_length=5, num_users=20, num_servers=5, num_apps=6)
servers, users, containers, apps = setup_sim(sim_param)

Initially run the scenario of job arrivals and process during small TS.

In [49]:
# Loop through big time step
bt = 0

# Update user location for big timestep

# Deploy the containers to the servers
cc = Central_Controller(servers, containers, sim_param, apps, users)
cc.VM_placement(users,apps,sim_param.deploy_rounds)

# For each small time step offload and serve at container
for st in range(sim_param.small_ts):
    # random order between users when offloading for each app
    usr_order = np.arange(len(users))
    np.random.shuffle(usr_order)
    
    temp_containers = {}
    queue_replies = {}
    
    # Make offloading decision
    for u in usr_order:
        # Generate load
        apps[u].new_load(ts_big=bt,ts_small=st)
        # Decide to offload given servers --> add offload policy to app class
        offload_u = apps[u].offload_uniform(cc.container_deployed, bt, st)
        for (s,a) in offload_u.keys():
            if (s,a) not in temp_containers:
                temp_containers[(s,a)] = np.empty([0,4])
            temp_containers[(s,a)] = np.append(temp_containers[(s,a)],offload_u[(s,a)],axis=0)
    
    # Scramble arrived job and add to queue
    for (s,a) in temp_containers.keys():
        sa_offload = temp_containers[(s,a)]
        np.random.shuffle(sa_offload) 
        replies = containers[(s,a)].add_to_queue(sa_offload)
        queue_replies[(s,a)] = replies
        
        


In [50]:
apps[0].offload_history

{(0, 0): array([0., 0., 0., 0., 1.]),
 (0, 1): array([ 0.,  0.,  0.,  0., 15.]),
 (0, 2): array([0., 0., 0., 0., 2.]),
 (0, 3): array([0., 0., 0., 0., 7.]),
 (0, 4): array([0., 0., 0., 0., 2.])}

In [51]:
offload_u.keys()

dict_keys([(0, 4), (1, 4), (3, 4), (4, 4)])

In [52]:
containers[(s,a)].queue

array([[18.,  0.,  1.,  1.],
       [12.,  0.,  1.,  1.],
       [12.,  1.,  1.,  1.],
       [12.,  2.,  3.,  3.],
       [18.,  3.,  2.,  2.],
       [18.,  4.,  1.,  1.],
       [12.,  0.,  1.,  1.],
       [18.,  2.,  2.,  2.],
       [12.,  3.,  1.,  1.],
       [12.,  4.,  1.,  1.],
       [18.,  4.,  2.,  2.]])

In [53]:
queue_replies

{(1,
  0): array([[ 1.        , 15.34622684],
        [ 4.        , 15.69108587],
        [11.        , 15.77730063],
        [10.        , 15.99283753],
        [ 7.        , 16.38080393]]),
 (0,
  1): array([[14.        ,  1.77789353],
        [ 2.        ,  2.01814941],
        [ 6.        ,  2.25840529],
        [17.        ,  2.45061   ]]),
 (1,
  1): array([[ 6.        , 17.44091952],
        [17.        , 17.5789743 ],
        [14.        , 17.80906558],
        [ 2.        , 18.08517513]]),
 (2,
  1): array([[ 2.        ,  1.5096534 ],
        [ 6.        ,  1.69375748],
        [17.        ,  1.80421992],
        [14.        ,  1.98832399]]),
 (3,
  1): array([[14.        ,  1.24757079],
        [17.        ,  1.34872518],
        [ 2.        ,  1.51731583],
        [ 6.        ,  1.68590648]]),
 (4,
  1): array([[ 6.        ,  1.62870218],
        [17.        ,  1.76075911],
        [14.        ,  2.02487298],
        [ 2.        ,  2.24496787]]),
 (4,
  3): array([[ 9.      

In [54]:
replies

array([[12.        ,  0.52541754],
       [18.        ,  0.60047719]])